本文使用相关系数的矩阵表达形式，实现了计算部分相关系数矩阵的加速算法，并实证检验了三种计算相关系数矩阵方法的运行速度。

- 在计算部分相关系数矩阵时，自定义的加速算法 相比 Pandas `.corr()` 方法提速约 2180 倍，比 Numpy `.corrcoef()` 方法提速约  115 倍。
- 在计算全部相关系数矩阵时，Numpy `.corrcoef()` 方法比自定义的加速算法略快 $10\%$，比 Pandas `.corr()` 方法快约 20 倍。

## 问题描述

我们需要计算若干列的相关系数矩阵的一部分。例如，原始数据有 10000 列、100 行，如果直接对这 10000 列数据计算相关系数矩阵，那么会得到一个 10000x10000 的矩阵，但我们只需要计算某 10 列与其他列的相关系数，那么我们只需要得到 10000x10 的矩阵即可。

第一种方法是，先计算出 $10000 \times 10000$ 的矩阵（这用 Pandas 的 `.corr()` 很容易实现），然后再取出我们需要的 $10000 \times 10$ 的矩阵。但这样做的问题是，计算出的 $10000 \times 10000$ 的矩阵的运算量非常大，这其中有很多值是我们不需要的，因此计算速度很慢。虽然 Numpy 中的 `.corrcoef()` 的计算速度比 Pandas 中的 `.corr()` 快很多，但是它仍然进行了很多不必要的运算。

第二种方法是，从相关系数的矩阵表达形式进行推导，手动用向量化运算得到我们需要的 10000x10 的矩阵。它的结果保证准确，且不会进行不必要的计算。

## 生成数据

本例中，我们生成一个 100 * 10000 的 Pandas 数据框，用于比较各种计算协方差矩阵方法的速度。

In [1]:
import pandas as pd
import numpy as np

In [2]:
np.random.seed(0)

df = pd.DataFrame(np.random.rand(100, 10000))
df

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,0.548814,0.715189,0.602763,0.544883,0.423655,0.645894,0.437587,0.891773,0.963663,0.383442,...,0.694983,0.194607,0.114037,0.262116,0.735502,0.550447,0.397151,0.758430,0.023787,0.813575
1,0.748268,0.180203,0.389023,0.037600,0.011788,0.996268,0.488197,0.372025,0.196172,0.807192,...,0.536808,0.528714,0.606949,0.705304,0.953551,0.748175,0.298267,0.446456,0.360127,0.625887
2,0.392173,0.041157,0.923301,0.406235,0.944282,0.722724,0.918320,0.823268,0.646897,0.067939,...,0.390984,0.763993,0.886611,0.535190,0.820328,0.622359,0.531140,0.852904,0.734230,0.638997
3,0.758125,0.503319,0.177017,0.832537,0.516825,0.926920,0.971807,0.675130,0.312100,0.575684,...,0.466430,0.770390,0.904429,0.317742,0.540895,0.668383,0.994911,0.588224,0.424615,0.901633
4,0.369256,0.211326,0.476905,0.082234,0.237659,0.083498,0.377752,0.231154,0.899234,0.172439,...,0.650715,0.877337,0.602423,0.426482,0.655736,0.546329,0.693310,0.233675,0.830924,0.551318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.128394,0.042076,0.952003,0.140991,0.955106,0.202784,0.672420,0.421446,0.364127,0.210888,...,0.479490,0.597782,0.831344,0.137618,0.149402,0.409812,0.220660,0.470112,0.230695,0.706102
96,0.015584,0.233506,0.239225,0.940945,0.503646,0.798759,0.737203,0.312909,0.422839,0.843037,...,0.536677,0.336206,0.540057,0.864079,0.458796,0.960221,0.507509,0.724927,0.273773,0.325543
97,0.204906,0.125667,0.680640,0.450485,0.607908,0.264120,0.060140,0.402881,0.638037,0.146196,...,0.646310,0.253233,0.619173,0.090566,0.916811,0.347903,0.508818,0.683824,0.030391,0.383813
98,0.575206,0.865496,0.640520,0.205309,0.905746,0.776942,0.977317,0.692063,0.374586,0.375532,...,0.877944,0.063698,0.177425,0.704477,0.262027,0.895913,0.446519,0.226104,0.526985,0.075002


## 使用不同方法计算相关系数矩阵

### 使用 Pandas 的 `.corr()` 方法计算相关系数矩阵

In [3]:
df.corr()

,0,1,2,3,4,5,6,7,8,9,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,1.000000,0.176915,0.000695,-0.074032,-0.131823,0.217309,0.048746,0.023913,-0.009316,0.120219,...,-0.072106,-0.015713,-0.138189,-0.023945,0.210072,-0.111980,0.071710,-0.273517,0.011651,0.011692
1,0.176915,1.000000,-0.093609,-0.036906,-0.042153,-0.033957,-0.074487,0.076879,0.054537,0.120812,...,-0.004401,-0.185725,-0.094619,0.139849,-0.076066,0.015955,-0.091318,-0.090413,-0.035353,-0.117072
2,0.000695,-0.093609,1.000000,0.026921,0.240980,-0.131894,0.075130,0.139438,0.110773,-0.159911,...,0.141726,0.122162,0.032242,0.005053,-0.138463,0.038591,0.010803,-0.008004,-0.073789,0.051757
3,-0.074032,-0.036906,0.026921,1.000000,0.120098,-0.140112,-0.007445,0.001608,0.067351,0.098839,...,-0.082701,0.012700,0.155302,-0.026240,-0.006347,0.097196,0.055631,0.278052,-0.117193,0.043521
4,-0.131823,-0.042153,0.240980,0.120098,1.000000,-0.148443,0.229543,-0.018978,0.053071,-0.097492,...,0.241090,-0.089095,0.035474,-0.074873,-0.105943,-0.064369,0.037381,0.039649,0.109127,0.079903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.111980,0.015955,0.038591,0.097196,-0.064369,-0.100968,0.124680,0.029289,0.085000,0.165217,...,0.015428,0.020980,0.023801,0.073931,-0.009990,1.000000,-0.051866,0.079591,-0.037915,-0.181415
9996,0.071710,-0.091318,0.010803,0.055631,0.037381,0.061301,-0.016855,0.032044,0.102410,0.049269,...,-0.108910,0.091617,-0.015837,0.001270,-0.090020,-0.051866,1.000000,0.004357,0.189117,-0.138257
9997,-0.273517,-0.090413,-0.008004,0.278052,0.039649,-0.020577,0.150100,-0.033782,0.079203,0.222427,...,-0.007391,-0.049121,0.004596,0.003603,-0.075922,0.079591,0.004357,1.000000,-0.102128,-0.082351
9998,0.011651,-0.035353,-0.073789,-0.117193,0.109127,0.108513,0.095505,-0.120213,-0.014490,-0.075557,...,-0.068107,-0.020685,-0.064335,-0.089839,-0.007462,-0.037915,0.189117,-0.102128,1.000000,-0.131022


如果我们只需要得到某 10 列与其他列的相关系数矩阵，那么我们只需要取出相关系数矩阵中的这 10 列即可。

In [4]:
result_pd = df.corr()[range(10)]
result_pd

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.176915,0.000695,-0.074032,-0.131823,0.217309,0.048746,0.023913,-0.009316,0.120219
1,0.176915,1.000000,-0.093609,-0.036906,-0.042153,-0.033957,-0.074487,0.076879,0.054537,0.120812
2,0.000695,-0.093609,1.000000,0.026921,0.240980,-0.131894,0.075130,0.139438,0.110773,-0.159911
3,-0.074032,-0.036906,0.026921,1.000000,0.120098,-0.140112,-0.007445,0.001608,0.067351,0.098839
4,-0.131823,-0.042153,0.240980,0.120098,1.000000,-0.148443,0.229543,-0.018978,0.053071,-0.097492
...,...,...,...,...,...,...,...,...,...,...
9995,-0.111980,0.015955,0.038591,0.097196,-0.064369,-0.100968,0.124680,0.029289,0.085000,0.165217
9996,0.071710,-0.091318,0.010803,0.055631,0.037381,0.061301,-0.016855,0.032044,0.102410,0.049269
9997,-0.273517,-0.090413,-0.008004,0.278052,0.039649,-0.020577,0.150100,-0.033782,0.079203,0.222427
9998,0.011651,-0.035353,-0.073789,-0.117193,0.109127,0.108513,0.095505,-0.120213,-0.014490,-0.075557


### 使用 Numpy 的 `.corrcoef()` 方法计算相关系数矩阵

Numpy 的 `.corrcoef()` 方法的调用中，需要注意将 `rowvar` 参数设置为 `False`，表示每一列为一个变量，每一行为一个观测值。

In [5]:
result_np = pd.DataFrame(np.corrcoef(df.values, rowvar=False)[:, :10])
result_np

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.176915,0.000695,-0.074032,-0.131823,0.217309,0.048746,0.023913,-0.009316,0.120219
1,0.176915,1.000000,-0.093609,-0.036906,-0.042153,-0.033957,-0.074487,0.076879,0.054537,0.120812
2,0.000695,-0.093609,1.000000,0.026921,0.240980,-0.131894,0.075130,0.139438,0.110773,-0.159911
3,-0.074032,-0.036906,0.026921,1.000000,0.120098,-0.140112,-0.007445,0.001608,0.067351,0.098839
4,-0.131823,-0.042153,0.240980,0.120098,1.000000,-0.148443,0.229543,-0.018978,0.053071,-0.097492
...,...,...,...,...,...,...,...,...,...,...
9995,-0.111980,0.015955,0.038591,0.097196,-0.064369,-0.100968,0.124680,0.029289,0.085000,0.165217
9996,0.071710,-0.091318,0.010803,0.055631,0.037381,0.061301,-0.016855,0.032044,0.102410,0.049269
9997,-0.273517,-0.090413,-0.008004,0.278052,0.039649,-0.020577,0.150100,-0.033782,0.079203,0.222427
9998,0.011651,-0.035353,-0.073789,-0.117193,0.109127,0.108513,0.095505,-0.120213,-0.014490,-0.075557


### 使用矩阵表达式计算相关系数矩阵

相关系数的矩阵表达形式的数学原理，可以参考 [Quickest way to calculate subset of correlation matrix - stackoverflow](https://stackoverflow.com/a/38195754/)。

In [6]:
def cal_corr(x, select_cols):
    n = x.shape[0]
    xsum = np.sum(x, 0, keepdims=True)
    xstd = np.std(x, 0, keepdims=True)

    return (
        np.dot(x.T, x[:, select_cols]) - np.dot(xsum.T, xsum[:, select_cols]) / n
    ) / (np.dot(xstd.T, xstd[:, select_cols]) * n)

In [7]:
result_cal_corr = pd.DataFrame(cal_corr(df.values, range(10)))
result_cal_corr

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.176915,0.000695,-0.074032,-0.131823,0.217309,0.048746,0.023913,-0.009316,0.120219
1,0.176915,1.000000,-0.093609,-0.036906,-0.042153,-0.033957,-0.074487,0.076879,0.054537,0.120812
2,0.000695,-0.093609,1.000000,0.026921,0.240980,-0.131894,0.075130,0.139438,0.110773,-0.159911
3,-0.074032,-0.036906,0.026921,1.000000,0.120098,-0.140112,-0.007445,0.001608,0.067351,0.098839
4,-0.131823,-0.042153,0.240980,0.120098,1.000000,-0.148443,0.229543,-0.018978,0.053071,-0.097492
...,...,...,...,...,...,...,...,...,...,...
9995,-0.111980,0.015955,0.038591,0.097196,-0.064369,-0.100968,0.124680,0.029289,0.085000,0.165217
9996,0.071710,-0.091318,0.010803,0.055631,0.037381,0.061301,-0.016855,0.032044,0.102410,0.049269
9997,-0.273517,-0.090413,-0.008004,0.278052,0.039649,-0.020577,0.150100,-0.033782,0.079203,0.222427
9998,0.011651,-0.035353,-0.073789,-0.117193,0.109127,0.108513,0.095505,-0.120213,-0.014490,-0.075557


### 检验各方法计算结果的一致性

In [8]:
np.isclose(result_pd, result_np).all()

True

In [9]:
np.isclose(result_pd, result_cal_corr).all()

True

## 比较各方法的计算速度

若只需要计算某 10 列与其他列的相关系数，那么：

- 使用矩阵表达式的方法的计算速度最快，因为它不会进行不必要的计算。
- 其次是 Numpy 的 `.corrcoef()` 方法。
- 最慢的是 Pandas 的 `.corr()` 方法。

In [10]:
%%timeit
df.corr()[range(10)]

10.9 s ± 85.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
%%timeit
pd.DataFrame(np.corrcoef(df.values, rowvar=False)[:, :10])

587 ms ± 22.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
pd.DataFrame(cal_corr(df.values, range(10)))

5.08 ms ± 334 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


若需要计算所有列之间的相关系数，那么：

- Numpy 的 `.corrcoef()` 方法的计算速度最快。
- 其次是矩阵表达式的方法。
- 最慢的是 Pandas 的 `.corr()` 方法。

In [13]:
%%timeit
df.corr()

11 s ± 92.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
%%timeit
pd.DataFrame(np.corrcoef(df.values, rowvar=False))

576 ms ± 14.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
pd.DataFrame(cal_corr(df.values, range(10000)))

674 ms ± 50.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
